# Implementación de pipelines con scikit-learn




Este notebook es una extensión de la introducción a scikit-learn. En este caso, veremos el mismo ejemplo de regresión, y uno de clasificación.



En esta sesión, aprenderemos cómo utilizar scikit-learn para construir una pipeline de Machine Learning completa. Cubriremos los siguientes pasos:

1. Carga y Exploración de Datos
2. Preprocesamiento de Datos
  - Normalización de variables numéricas
  - Codificación de variables categóricas
3. Selección del Modelo y Función de Pérdida
4. Validación Cruzada y Selección de Hiperparámetros
5. Evaluación del Modelo
  - RMSE para Regresión
  - Accuracy para Clasificación

## 1. Recolección de Datos
Carga y Exploración de Datos

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score


Carga de datos para el ejemplo de regresión

In [ ]:
from sklearn.datasets import fetch_california_housing

# Cargar el conjunto de datos California Housing
housing = fetch_california_housing()
X_reg = pd.DataFrame(housing.data, columns=housing.feature_names)
y_reg = pd.Series(housing.target, name='price')

# Mostrar las primeras filas
print("Datos de Regresión (California Housing):")
display(X_reg.head())
display(y_reg.head())

print("Descripción del Conjunto de Datos:")
print(housing.DESCR)


Datos de Regresión (California Housing):


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


,price
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422


Descripción del Conjunto de Datos:
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 199

Carga de datos para el ejemplo de clasificación

In [ ]:
from sklearn.datasets import load_iris

# Cargar el conjunto de datos Iris
iris = load_iris()
X_clf = pd.DataFrame(iris.data, columns=iris.feature_names)
y_clf = pd.Series(iris.target, name='Species')

# Mostrar las primeras filas
print("Datos de Clasificación (Iris):")
display(X_clf.head())
display(y_clf.head())

print("Descripción del Conjunto de Datos:")
print(iris.DESCR)


Datos de Clasificación (Iris):


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


,Species
0,0
1,0
2,0
3,0
4,0


Descripción del Conjunto de Datos:
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 199

## 2. Limpieza, Preparación y División de Datos

Esto incluye:


*   **Limpieza de Datos**: no vamos a hacerlo en este ejemplo
*   **Transformación de Datos**:
  *   Variables numéricas: normalización
  *   Variables categóricas: codificación con One-Hot-Encoding
*   **Ingeniería de Características**: no vamos a hacerlo en este ejemplo
*   **División del dataset**: haremos cross-validation en el paso 3, así que aquí solo dividiremos en training y test. Cuando hagamos cross-validation, se subdividirá el training.

### Transformación de datos

Primero, identificamos qué variables son numéricas y cuáles categóricas

In [ ]:
numeric_features_reg = X_reg.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features_reg = X_reg.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
numeric_features_clf = X_clf.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features_clf = X_clf.select_dtypes(include=['object', 'category']).columns.tolist()

Definir Preprocesadores

In [ ]:
# Preprocesador para Regresión
numeric_transformer_reg = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer_reg = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_reg = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_reg, numeric_features_reg),
        ('cat', categorical_transformer_reg, categorical_features_reg)
    ])


In [ ]:
# Preprocesador para Clasificación
numeric_transformer_clf = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer_clf = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor_clf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_clf, numeric_features_clf),
        ('cat', categorical_transformer_clf, categorical_features_clf)
    ])


### División del dataset

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y test
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=0)

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y test
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X_clf, y_clf, test_size=0.3, random_state=0)

## Selección del modelo, entrenamiento, validación y ajuste de hiper parámetros

### Selección del Modelo (y Función de Pérdida)

Dependiendo del problema (regresión o clasificación), seleccionaremos un modelo apropiado y definiremos su función de pérdida.

Modelos para Regresión

- Regresión Lineal y Polinómica: Minimiza el error cuadrático medio (MSE).

Modelos para Clasificación

- Regresión Logística: Minimiza la entropía cruzada.
- K-Nearest Neighbors Classifier (KNN Classifier): Minimiza la distancia entre las predicciones y las clases reales.


### Validación Cruzada y Ajuste de Hiperparámetros

Nótese que usamos el método `GridSearchCV`: Este método hace una GridSearch para buscar los mejores hiper parámetros. Además, cada entrenamiento lo hace con cross-validation.

Por ejemplo, en regresión probamos 3 hiper parámetros y especificamos cross-validation con 5 folds, así que se hacen 15 entrenamientos (5 para cada hiperparámetro). Además, después de encontrar la mejor combinación de hiperparámetros, reentrena un modelo con el conjunto de training entero, así que finalmente se hacen 16 entrenamientos. Ese modelo es el que después se usará para hacer predicciones.

Pipeline para regresión

In [ ]:
# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train_reg.shape}")
print(f"Tamaño del conjunto de test: {X_test_reg.shape}")

# Definir el pipeline para regresión con regresión polinómica
pipeline_reg = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('poly', PolynomialFeatures()), # Para hacer regresión polinómica, hay que definir este preprocesador
    ('regressor', LinearRegression())
])

# Definir la cuadrícula de hiperparámetros para GridSearch
param_grid_reg = {
    'poly__degree': [1, 2, 3],  # Grados del polinomio
}

# Configurar GridSearchCV
grid_search_reg = GridSearchCV(
    pipeline_reg,
    param_grid_reg,
    cv=5, # 5-fold
    scoring='neg_mean_squared_error',
    refit=True,
)

# Ajustar el modelo
grid_search_reg.fit(X_train_reg, y_train_reg)

# Inspeccionar los resultados de la cross-validation
cv_results = pd.DataFrame(grid_search_reg.cv_results_)
print("Resultados de la Cross-Validation:")
print(cv_results[['param_poly__degree', 'mean_test_score', 'std_test_score']])

# Mostrar los mejores parámetros
print("Mejores parámetros para Regresión:")
print(grid_search_reg.best_params_)


Tamaño del conjunto de entrenamiento: (16512, 8)
Tamaño del conjunto de test: (4128, 8)
Resultados de la Cross-Validation:
   param_poly__degree  mean_test_score  std_test_score
0                   1        -0.528176        0.016725
1                   2        -3.936059        4.430414
2                   3    -85312.929338   169700.864061
Mejores parámetros para Regresión:
{'poly__degree': 1}


Pipeline para Clasificación

In [ ]:
# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train_clf.shape}")
print(f"Tamaño del conjunto de test: {X_test_clf.shape}")


# Definir el pipeline para clasificación
pipeline_clf = Pipeline(steps=[
    ('preprocessor', preprocessor_clf),
    ('classifier', KNeighborsClassifier())
])

# Definir la cuadrícula de hiperparámetros para GridSearch
param_grid_clf = {
    'classifier': [
        # LogisticRegression(max_iter=1000),
        KNeighborsClassifier()],
    # 'classifier__C': [0.1, 1.0, 10.0],  # Solo aplicable para Logistic Regression
    'classifier__n_neighbors': [5, 7, 15]  # Solo aplicable para KNN
}

# Configurar GridSearchCV
grid_search_clf = GridSearchCV(
    pipeline_clf,
    param_grid_clf,
    cv=5,
    scoring='accuracy',
    refit=True,
    n_jobs=-1)

# Ajustar el modelo
grid_search_clf.fit(X_train_clf, y_train_clf)

# Inspeccionar los resultados de la cross-validation
cv_results = pd.DataFrame(grid_search_clf.cv_results_)
print("Resultados de la Cross-Validation:")
print(cv_results[['param_classifier__n_neighbors', 'mean_test_score', 'std_test_score']])


# Mostrar los mejores parámetros
print("Mejores parámetros para Clasificación:")
print(grid_search_clf.best_params_)


Tamaño del conjunto de entrenamiento: (105, 4)
Tamaño del conjunto de test: (45, 4)
Resultados de la Cross-Validation:
   param_classifier__n_neighbors  mean_test_score  std_test_score
0                              5         0.952381        0.073771
1                              7         0.933333        0.071270
2                             15         0.942857        0.055533
Mejores parámetros para Clasificación:
{'classifier': KNeighborsClassifier(), 'classifier__n_neighbors': 5}


Nota: En este ejemplo, algunos hiperparámetros no son aplicables a ciertos modelos. Por simplicidad, elegimos solo el KNN y su hiper-parámetro `n_neighbors`

## 4. Evaluación del Modelo

Evaluación para Regresión

In [ ]:
# Predecir en el conjunto de test
y_pred_reg = grid_search_reg.predict(X_test_reg)

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
print(f"RMSE en el conjunto de test: {rmse:.2f}")


RMSE en el conjunto de test: 0.73


Nota: `grid_search_reg.predict()` usa un modelo entrenado con todo el conjunto de training y la mejor combinación de hiper parámetros. Se puede acceder a ese modelo con `grid_search_clf.best_estimator_`

Evaluación para Clasificación


In [ ]:
# Predecir en el conjunto de test
y_pred_clf = grid_search_clf.predict(X_test_clf)

# Calcular Accuracy
accuracy = accuracy_score(y_test_clf, y_pred_clf)
print(f"Accuracy en el conjunto de test: {accuracy:.2f}")


Accuracy en el conjunto de test: 0.98


### Inspección de las predicciones

In [ ]:
# Mostrar algunas predicciones junto con los valores reales
results = pd.DataFrame({'Actual': y_test_clf, 'Predicted': y_pred_clf})
print(results.head(10))


     Actual  Predicted
114       2          2
62        1          1
33        0          0
107       2          2
7         0          0
100       2          2
40        0          0
86        1          1
76        1          1
71        1          1
